<center>
    <img src="https://i.imgur.com/HRhd2Y0.png" width="700" height="500">
</center>

In [ ]:
# Import required libraries
import pandas as pd
from functions import find_csv_files, load_csvs_to_dict, sort_and_classify_column, transform_to_days

# Set pandas display option for maximum columns
pd.set_option('display.max_columns', 50)

# Define path to data files
DATA_PATH = "/workspaces/OpenClassroom--Machine-Learning-Engineer/P4/data/"
DATA_SAVE_FILE = "data_cleaned.csv.gz"

# Find all CSV files in the data path
csv_files = find_csv_files(DATA_PATH)

# Load CSV files into a dictionary of DataFrames
dfs = load_csvs_to_dict(csv_files)

### RFM (Recency, Frequency, Monetary) Method Analysis ###

# --- Recency DataFrame ---
# Sort and classify the 'order_purchase_timestamp' column and transform it to days
recency_df = sort_and_classify_column(dfs["olist_orders_dataset"], 
                                      column_name="order_purchase_timestamp",
                                      datetime=True)

recency_df["Recency"] = transform_to_days(recency_df["order_purchase_timestamp"])

# Drop unnecessary columns
columns_to_drop = ["order_approved_at", 
                   "order_id", 
                   "order_delivered_carrier_date", 
                   "order_purchase_timestamp", 
                   "order_delivered_customer_date", 
                   "order_estimated_delivery_date"]

recency_df.drop(columns=columns_to_drop, inplace=True)

# --- Frequency DataFrame ---
# Merge orders and customers DataFrames, then count the number of orders per unique customer
frequency_df = dfs["olist_orders_dataset"].merge(dfs["olist_customers_dataset"], 
                                                 on="customer_id")
frequency_df = frequency_df.groupby(by="customer_unique_id").count().reset_index().loc[:, ["customer_unique_id", "order_id"]]

# Rename the 'order_id' column to 'Frequency'
frequency_df.rename(columns={"order_id": "Frequency"}, inplace=True)

# --- Note ---
# sort_and_classify_column is basically applying the rfm method
# Ordered from lowest to highest
# Binning applied such that three classes are produced
# Low, Medium & High
# This method is applied to recency, frequency & monetary columns
frequency_df = sort_and_classify_column(frequency_df, column_name="Frequency")

# --- Monetary DataFrame ---
# Calculate the total payment per order
per_order_payment = dfs["olist_order_payments_dataset"].groupby(by="order_id").sum().reset_index()

# Merge with orders DataFrame to get the monetary value
monetary_df = dfs["olist_orders_dataset"].merge(per_order_payment, 
                                                on="order_id")

monetary_df = sort_and_classify_column(monetary_df, 
                                       column_name="payment_value")

# Drop unnecessary columns
columns_to_drop = ["order_approved_at", 
                   "order_status", 
                   "order_delivered_carrier_date", 
                   "order_delivered_customer_date", 
                   "order_estimated_delivery_date"]

monetary_df.drop(columns=columns_to_drop, inplace=True)

# --- Create the Final RFM DataFrame ---
# Merge all the individual Recency, Frequency, and Monetary DataFrames
rfm_df = dfs["olist_customers_dataset"].merge(recency_df, how="left", on="customer_id") \
                                       .merge(frequency_df, how="left", on="customer_unique_id") \
                                       .merge(monetary_df, how="left", on="customer_id")

### Additional Features: Basket Information ###

# --- Number of Items per Order ---
df_items_per_order = dfs["olist_order_items_dataset"].groupby("order_id").count().reset_index().rename(columns={"product_id": "number_of_products"})

# --- Mean Product Price and Freight Value ---
df_mean_product_price = dfs["olist_order_items_dataset"].groupby("product_id").mean().reset_index()[["product_id",
                                                                                                      "price", 
                                                                                                      "freight_value"]]

# --- Product Measurements ---
df_product_measurements = dfs["olist_products_dataset"][["product_id", 
                                                         "product_weight_g", 
                                                         "product_length_cm", 
                                                         "product_height_cm", 
                                                         "product_width_cm"]]

# --- Merge Product Price and Measurements ---
df_product_price_measures = df_mean_product_price.merge(df_product_measurements, 
                                                        on="product_id", 
                                                        how="left")

# --- Merge Product and Order Information ---
df_product_order_info = dfs["olist_order_items_dataset"][["order_id", 
                                                          "product_id"]].merge(df_product_price_measures, 
                                                                                           on="product_id", 
                                                                                           how="left")

# --- Calculate Basket Information ---
df_basket_info = df_product_order_info.drop(columns=["product_id"]).groupby("order_id").mean().reset_index().merge(df_items_per_order, 
                                                                                                                   on="order_id", 
                                                                                                                   how="right")

# --- Merge Basket Information with RFM DataFrame ---
final_df = rfm_df.merge(df_basket_info, 
                        how="left", 
                        on="order_id")

# Display the final DataFrame
final_df

/tmp/ipykernel_5503/2611050141.py:50: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  per_order_payment = dfs["olist_order_payments_dataset"].groupby(by="order_id").sum().reset_index()
/tmp/ipykernel_5503/2611050141.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_mean_product_price = dfs["olist_order_items_dataset"].groupby("product_id").mean().reset_index()[["product_id",


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_status,order_purchase_timestamp_class,Recency,Frequency,Frequency_class,order_id,order_purchase_timestamp,payment_sequential,payment_installments,payment_value,payment_value_class,price_x,freight_value_x,product_weight_g,product_length_cm,product_height_cm,product_width_cm,order_item_id,number_of_products,seller_id,shipping_limit_date,price_y,freight_value_y
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,delivered,Low,2351,1,Low,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,1.0,2.0,146.87,Low,119.362881,31.543220,8683.0,54.0,64.0,31.0,1.0,1.0,1.0,1.0,1.0,1.0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,delivered,Medium,2109,1,Low,29150127e6685892b6eab3eec79f59c7,2018-01-12 20:48:24,1.0,8.0,335.48,Low,291.185319,41.179787,10150.0,89.0,15.0,40.0,1.0,1.0,1.0,1.0,1.0,1.0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,delivered,High,1983,1,Low,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,1.0,7.0,157.73,Low,143.958000,42.509000,8267.0,52.0,52.0,17.0,1.0,1.0,1.0,1.0,1.0,1.0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,delivered,High,2050,1,Low,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,1.0,1.0,173.30,Low,164.758095,37.736667,12160.0,56.0,51.0,28.0,1.0,1.0,1.0,1.0,1.0,1.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,delivered,High,1912,1,Low,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07-29 09:51:30,1.0,8.0,252.25,Low,230.000000,35.017500,5200.0,45.0,15.0,35.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,delivered,High,2025,1,Low,6760e20addcf0121e9d58f2f1ff14298,2018-04-07 15:48:17,1.0,6.0,88.78,Low,78.900000,17.278000,611.0,22.0,22.0,23.0,1.0,1.0,1.0,1.0,1.0,1.0
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,delivered,High,2028,1,Low,9ec0c8947d973db4f4e8dcf1fbfa8f1b,2018-04-04 08:20:22,1.0,3.0,129.06,Low,106.914286,24.420000,1211.0,25.0,24.0,22.0,1.0,1.0,1.0,1.0,1.0,1.0
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,delivered,High,2023,1,Low,fed4434add09a6f332ea398efd656a5c,2018-04-08 20:11:50,1.0,5.0,56.04,Low,37.000000,18.050000,870.0,25.0,20.0,18.0,1.0,1.0,1.0,1.0,1.0,1.0
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,delivered,Medium,2179,1,Low,e31ec91cea1ecf97797787471f98a8c2,2017-11-03 21:08:33,1.0,2.0,711.07,Low,644.988444,31.671333,710.0,19.0,13.0,14.0,1.0,1.0,1.0,1.0,1.0,1.0


In [3]:
# Dictionary to map old column names to new column names
rename_dict = {
    'customer_id': 'CustomerID',
    'customer_unique_id': 'UniqueCustomerID',
    'customer_zip_code_prefix': 'CustomerZipCodePrefix',
    'customer_city': 'CustomerCity',
    'customer_state': 'CustomerState',
    'order_status': 'OrderStatus',
    'order_purchase_timestamp_class': 'RecencyClass',
    'Recency': 'Recency',
    'Frequency': 'Frequency',
    'Frequency_class': 'FrequencyClass',
    'order_id': 'OrderID',
    'order_purchase_timestamp': 'OrderTimestamp',
    'payment_sequential': 'PaymentSequential',
    'payment_installments': 'PaymentInstallments',
    'payment_value': 'TotalPaymentValue',
    'payment_value_class': 'TotalPaymentValueClass',
    'price_x': 'MeanProductPrice',
    'freight_value_x': 'MeanFreightValue',
    'product_weight_g': 'AverageProductWeightG',
    'product_length_cm': 'AverageProductLengthCM',
    'product_height_cm': 'AverageProductHeightCM',
    'product_width_cm': 'AverageProductWidthCM',
    'order_item_id': 'OrderItemID',
    'number_of_products': 'NumberOfProductsInOrder',
    'seller_id': 'SellerID',
    'shipping_limit_date': 'ShippingLimitDate',
    'price_y': 'IndividualProductPrice',
    'freight_value_y': 'IndividualFreightValue'
}

# Final Columns to drop
columns_to_drop = ["PaymentSequential","OrderItemID", "SellerID","ShippingLimitDate", "IndividualProductPrice", "IndividualFreightValue"]

# Rename the DataFrame columns
final_df.rename(columns=rename_dict, inplace=True)

# Drop Columns
final_df.drop(columns=columns_to_drop, inplace = True)

# Drop missing rows 
final_df.dropna(inplace=True)

# Quick Check
final_df

,CustomerID,UniqueCustomerID,CustomerZipCodePrefix,CustomerCity,CustomerState,OrderStatus,RecencyClass,Recency,Frequency,FrequencyClass,OrderID,OrderTimestamp,PaymentInstallments,TotalPaymentValue,TotalPaymentValueClass,MeanProductPrice,MeanFreightValue,AverageProductWeightG,AverageProductLengthCM,AverageProductHeightCM,AverageProductWidthCM,NumberOfProductsInOrder
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,delivered,Low,2351,1,Low,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,2.0,146.87,Low,119.362881,31.543220,8683.0,54.0,64.0,31.0,1.0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,delivered,Medium,2109,1,Low,29150127e6685892b6eab3eec79f59c7,2018-01-12 20:48:24,8.0,335.48,Low,291.185319,41.179787,10150.0,89.0,15.0,40.0,1.0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,delivered,High,1983,1,Low,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,7.0,157.73,Low,143.958000,42.509000,8267.0,52.0,52.0,17.0,1.0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,delivered,High,2050,1,Low,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,1.0,173.30,Low,164.758095,37.736667,12160.0,56.0,51.0,28.0,1.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,delivered,High,1912,1,Low,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07-29 09:51:30,8.0,252.25,Low,230.000000,35.017500,5200.0,45.0,15.0,35.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,delivered,High,2025,1,Low,6760e20addcf0121e9d58f2f1ff14298,2018-04-07 15:48:17,6.0,88.78,Low,78.900000,17.278000,611.0,22.0,22.0,23.0,1.0
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,delivered,High,2028,1,Low,9ec0c8947d973db4f4e8dcf1fbfa8f1b,2018-04-04 08:20:22,3.0,129.06,Low,106.914286,24.420000,1211.0,25.0,24.0,22.0,1.0
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,delivered,High,2023,1,Low,fed4434add09a6f332ea398efd656a5c,2018-04-08 20:11:50,5.0,56.04,Low,37.000000,18.050000,870.0,25.0,20.0,18.0,1.0
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,delivered,Medium,2179,1,Low,e31ec91cea1ecf97797787471f98a8c2,2017-11-03 21:08:33,2.0,711.07,Low,644.988444,31.671333,710.0,19.0,13.0,14.0,1.0


In [3]:
# Save Dataframe
final_df.to_csv(DATA_PATH + DATA_SAVE_FILE, compression="gzip", index = False)
